In [6]:
model_to_special_token = {
        "meta-llama/Llama-3.2-3B-Instruct": "<|finetune_right_pad_id|>", # 128004,
        "google/gemma-7b-it": 10,
        "mistralai/Ministral-8B-Instruct-2410": 20,
        "meta-llama/Llama-3.1-8B-Instruct": 128002
    }

model_id = "meta-llama/Llama-3.1-8B-Instruct"

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
sep_tok_id = model_to_special_token[model_id]

In [8]:
def get_separator_ids_automatic(tokenizer, sep_tok_id=None):
    if sep_tok_id is None:
        if tokenizer.unk_token_id is not None:
            sep_tok_id = tokenizer.unk_token_id
        else:
            sep_tok_id = 128002 # for Llama 3.1

    sep_tok_str = tokenizer.decode(sep_tok_id)

    messages = [
                {"role": "user", "content": sep_tok_str}, 
                {"role": "assistant", "content": sep_tok_str}
    ]

    base_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=False
    )

    # Split base_ids at sep_tok_id occurrences
    split_indices = [i for i, token in enumerate(base_ids) if token == sep_tok_id]
    assert len(split_indices) == 2, f"Expected 2 occurrences of sep_tok_id but found {len(split_indices)}. Please look at the hf tokenizer_config.json file to find a better separator token than {sep_tok_id}"
    
    # Get IDs between the two separators (excluding separators)
    ids_between = base_ids[split_indices[0]+1:split_indices[1]]
    return ids_between


separator_ids = get_separator_ids_automatic(tokenizer, sep_tok_id)

In [9]:
separator_ids

[128009, 128006, 78191, 128007, 271]

In [10]:
tokenizer.decode(separator_ids)

'<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'